In [13]:
import numpy as np
import random

from models.Coordinate import Coordinate
import config
import matplotlib.pyplot as plt


In [14]:
def read_data(file_path):
    list_cities = [None]

    with open(file_path, 'r') as f:
        data = f.readlines()
        NUMBER_CITIES = int(data[3].split(':')[-1].strip())

        for line in data[6: 6 + NUMBER_CITIES]:
            _, x, y = list(map(int, line.split()))
            list_cities.append(Coordinate(x, y))

    return list_cities, NUMBER_CITIES


list_cities, NUMBER_CITIES = read_data(config.DATA_PATH)
print(list_cities, NUMBER_CITIES)


[None, Coordinate: x = 37, y = 52, Coordinate: x = 49, y = 49, Coordinate: x = 52, y = 64, Coordinate: x = 20, y = 26, Coordinate: x = 40, y = 30, Coordinate: x = 21, y = 47, Coordinate: x = 17, y = 63, Coordinate: x = 31, y = 62, Coordinate: x = 52, y = 33, Coordinate: x = 51, y = 21, Coordinate: x = 42, y = 41, Coordinate: x = 31, y = 32, Coordinate: x = 5, y = 25, Coordinate: x = 12, y = 42, Coordinate: x = 36, y = 16, Coordinate: x = 52, y = 41, Coordinate: x = 27, y = 23, Coordinate: x = 17, y = 33, Coordinate: x = 13, y = 13, Coordinate: x = 57, y = 58, Coordinate: x = 62, y = 42, Coordinate: x = 42, y = 57, Coordinate: x = 16, y = 57, Coordinate: x = 8, y = 52, Coordinate: x = 7, y = 38, Coordinate: x = 27, y = 68, Coordinate: x = 30, y = 48, Coordinate: x = 43, y = 67, Coordinate: x = 58, y = 48, Coordinate: x = 58, y = 27, Coordinate: x = 37, y = 69, Coordinate: x = 38, y = 46, Coordinate: x = 46, y = 10, Coordinate: x = 61, y = 33, Coordinate: x = 62, y = 63, Coordinate: x = 

In [15]:
def init_population():
    population = [None]

    for _ in range(1, config.POPULATION_SIZE + 1):
        while True:
            individual = random.sample(
                range(1, NUMBER_CITIES + 1), NUMBER_CITIES)
            if individual not in population:
                break
        population.append(individual)

    return population


population = init_population()
print(population)


[None, [40, 45, 8, 1, 30, 18, 6, 12, 34, 5, 20, 14, 19, 2, 21, 15, 32, 47, 22, 10, 41, 42, 26, 31, 39, 50, 51, 7, 29, 27, 24, 36, 38, 33, 11, 3, 23, 16, 4, 13, 44, 35, 37, 49, 48, 17, 28, 9, 25, 46, 43], [17, 11, 40, 33, 5, 12, 7, 35, 2, 23, 47, 20, 1, 46, 9, 15, 14, 44, 37, 32, 31, 51, 38, 10, 4, 6, 21, 22, 26, 25, 49, 27, 13, 39, 19, 29, 34, 43, 36, 24, 42, 16, 18, 50, 8, 41, 28, 30, 3, 48, 45], [35, 14, 25, 39, 44, 38, 51, 13, 37, 36, 20, 41, 29, 22, 46, 32, 27, 7, 6, 1, 15, 12, 33, 18, 50, 5, 17, 42, 4, 21, 9, 3, 23, 28, 19, 30, 48, 10, 40, 31, 2, 24, 43, 11, 26, 45, 47, 16, 34, 8, 49], [48, 17, 40, 8, 44, 26, 25, 37, 2, 49, 38, 46, 35, 14, 45, 18, 39, 24, 51, 16, 42, 4, 20, 43, 50, 1, 11, 3, 33, 29, 31, 7, 12, 15, 19, 23, 32, 5, 41, 28, 13, 21, 27, 36, 30, 22, 34, 9, 10, 47, 6], [4, 20, 10, 49, 23, 8, 7, 21, 31, 25, 6, 27, 17, 46, 19, 43, 42, 37, 22, 1, 40, 15, 34, 28, 14, 47, 29, 3, 35, 33, 39, 24, 5, 12, 50, 13, 48, 26, 2, 41, 9, 32, 45, 38, 44, 18, 36, 51, 30, 11, 16], [17, 48,

In [16]:
DISTANCE_MATRIX = [[None] * (NUMBER_CITIES + 1)] * (NUMBER_CITIES + 1)


def euclidean_distance(index_city1, index_city2):
    if not DISTANCE_MATRIX[index_city1][index_city2]:
        DISTANCE_MATRIX[index_city1][index_city2] = np.linalg.norm((list_cities[index_city1].x - list_cities[index_city2].x,
                                                                    list_cities[index_city1].y - list_cities[index_city2].y))

    return DISTANCE_MATRIX[index_city1][index_city2]


def cost(individual):
    if not individual:
        return 0
    c = 0

    for i in range(len(individual) - 1):
        c += euclidean_distance(individual[i], individual[i+1])
    # Last city with first city
    c += euclidean_distance(individual[-1], individual[0])

    return c


def find_best(population):
    best_individual = population[1]
    best_cost = cost(population[1])
    for individual in population[2:]:
        tmp = cost(individual)
        if tmp < best_cost:
            best_individual = individual
            best_cost = tmp
            
    return best_individual, best_cost


In [17]:
def select_parents(population):
    candidates = random.sample(population[1:], 4)

    return (candidates[0] if cost(candidates[0]) < cost(candidates[1]) else candidates[1],
            candidates[2] if cost(candidates[2]) < cost(candidates[3]) else candidates[3])


def crossover(parents):
    if random.random() < config.P_CROSSOVER:
        pos1, pos2 = tuple(random.sample(range(len(parents[0])), 2))
        if pos1 > pos2:
            tmp = pos1
            pos1 = pos2
            pos2 = tmp

        children = []
        for parent in parents:
            child = parent[pos1: pos2 + 1]
            tmp = []
            for i in range(pos2 + 1, pos2 + 1 + len(parent)):
                if i >= len(parent):
                    i %= len(parent)
                if parent[i] not in child:
                    tmp.append(parent[i])
            

            child = (tmp[-pos1:] if pos1 != 0 else []) + child + tmp[:len(tmp) - pos1]
            assert len(child) == len(parent)
            
            children.append(child)
            # print(children)
        return tuple(children)

    return None, None


def mutation(parents):
    if random.random() < config.P_MUTATION:
        children = []
        for parent in parents:
            pos1, pos2 = tuple(random.sample(range(len(parent)), 2))
            if pos1 > pos2:
                tmp = pos1
                pos1 = pos2
                pos2 = tmp
            
            parent[pos1], parent[pos2] = parent[pos2], parent[pos1]
            children.append(parent)
        return tuple(children)

    return None, None


def gen_offspring(population):
    offspring = [None]

    while len(offspring) < len(population):
        parents = select_parents(population)

        children = crossover(parents)
        if children[0]:
            for child in children:
                offspring.append(child)
            # Mutation if only crossover
            children = mutation(parents)
            if children[0]:
                for child in children:
                    offspring.append(child)
    
    assert len(offspring) >= len(population)
    random_index_remove = random.randint(0, len(offspring) - 1)
    offspring[random_index_remove] = find_best(population)[0]

    return offspring


In [18]:
history = [None]

for i in range(config.NUM_GENERATIONS):
    history.append(find_best(population))
    offspring = gen_offspring(population)
    population = offspring


KeyboardInterrupt: 

In [ ]:
plt.title("Line graph")
plt.xlabel("X axis")
plt.ylabel("Y axis")
plt.plot(range(1, len(history)), list(map(lambda x: x[1], history[1:])), color ="red")
plt.show()
print(list(map(lambda x: x[1], history)))